In [ ]:
import pandas
import numpy as np
import ipywidgets
import pickle
from IPython.display import display
from ipywidgets import VBox, HBox, Button

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

TOPICS = [
    "Crime",
    "Parking",
    "Transit",
    "Homelessness",
    "Schools",
    "Environment",
    "Affordability",
    "Development",
    "Density",
    "Politics",
    "Gentrification",
]

In [ ]:
df2011 = pandas.read_csv('data/2011_Housing_Inventory_latlng_zip.csv')
df2012 = pandas.read_csv('data/2012_Housing_Inventory_latlng.csv')
df2013 = pandas.read_csv('data/2013_Housing_Inventory_latlng.csv')
df2014 = pandas.read_csv('data/2014_Housing_Inventory_latlng.csv')
df_map = {2011: df2011, 2012: df2012, 2013: df2013, 2014: df2014}

In [ ]:
datasets = [(df2011, 'STRDADDRESS'), (df2012, 'STDADDRESS'), (df2013, 'STDADDRESS'), (df2014, 'STDADD')]
latlon_lookup = dict()

for dataset, addcol in datasets:
    for addr, latlon in zip(dataset[addcol], dataset.LAT_LNG):
        if latlon == '[]':
            continue
            
        latlng = latlon[1:-1].split(', ')
        lat = float(latlng[0])
        lng = float(latlng[1])
        latlon_lookup[addr.lower()] = (lat, lng)

In [ ]:
topics_data = pickle.load(open('data/topics_data.pkl', 'rb'))

In [ ]:
def create_circles_for_topic(m, data, topic):
    circles = []
    
    for address, adddata in data.items():
        if address.lower() not in latlon_lookup:
            continue
        if isinstance(adddata['topic'], int) and adddata['topic'] == 0:
            continue
            
        lat, lon = latlon_lookup[address.lower()]
        if sum(adddata['topic']) == 0:
            topic_score = float(adddata['topic'][topic])
        else:
            topic_score = float(adddata['topic'][topic]) / sum(adddata['topic'])
        
        g = min(int(topic_score*255*5), 255)
        color = "#{:0>2x}{:0>2x}{:0>2x}".format(g, g, 0)
        fill_color = color

        c = Circle(location=(lat,lon), radius=50, color=color, fill_color=fill_color, opacity=1.0)
        circles.append(c)
        
    # add all circles at once. HACK!!
    for layer in circles:
        layer._map = m
    m.layers = tuple([l for l in m.layers] + circles)
    for layer in circles:
        layer.visible = True
        
    return circles

In [ ]:
def on_slider_change(w, x):
    print x
    global circles
    m.layers = tuple([m.layers[0]])
    circles = create_circles_for_topic(m, topics_data, x)

In [ ]:
center = [37.76284413400416, -122.39490509033203]
zoom = 12
m = Map(default_tiles=TileLayer(opacity=1.0, url='https://tiles.wmflabs.org/bw-mapnik/{z}/{x}/{y}.png'), center=center, zoom=zoom)
create_circles_for_topic(m, topics_data, 0)
display(m)
buttons = [Button(description=x) for x in TOPICS]
indices = [range(len(buttons))]

buttons[0].on_click(lambda x: on_slider_change(x, 0))
buttons[1].on_click(lambda x: on_slider_change(x, 1))
buttons[2].on_click(lambda x: on_slider_change(x, 2))
buttons[3].on_click(lambda x: on_slider_change(x, 3))
buttons[4].on_click(lambda x: on_slider_change(x, 4))
buttons[5].on_click(lambda x: on_slider_change(x, 5))
buttons[6].on_click(lambda x: on_slider_change(x, 6))
buttons[7].on_click(lambda x: on_slider_change(x, 7))
buttons[8].on_click(lambda x: on_slider_change(x, 8))
buttons[9].on_click(lambda x: on_slider_change(x, 9))
buttons[10].on_click(lambda x: on_slider_change(x, 10))

VBox([HBox(buttons[:6]), HBox(buttons[6:])])